[Bag of words](https://openclassrooms.com/fr/courses/4470541-analysez-vos-donnees-textuelles/4855001-representez-votre-corpus-en-bag-of-words)

[Feature extraction](https://scikit-learn.org/stable/modules/feature_extraction.html) ; [FE - Count extraction](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())

print(X.toarray())
print vectorizer.vocabulary_.get('the')


vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(2, 2))
X2 = vectorizer2.fit_transform(corpus)
print(vectorizer2.get_feature_names())

print(X2.toarray())


[u'and', u'document', u'first', u'is', u'one', u'second', u'the', u'third', u'this']
[[0 1 1 1 0 0 1 0 1]
 [0 2 0 1 0 1 1 0 1]
 [1 0 0 1 1 0 1 1 1]
 [0 1 1 1 0 0 1 0 1]]
6
[u'and this', u'document is', u'first document', u'is the', u'is this', u'second document', u'the first', u'the second', u'the third', u'third one', u'this document', u'this is', u'this the']
[[0 0 1 1 0 0 1 0 0 0 0 1 0]
 [0 1 0 1 0 1 0 1 0 0 1 0 0]
 [1 0 0 1 0 0 0 0 1 1 0 1 0]
 [0 0 1 0 1 0 1 0 0 0 0 0 1]]


[Stemming, stopword... & NLTK](https://openclassrooms.com/fr/courses/4470541-analysez-vos-donnees-textuelles/4854971-nettoyez-et-normalisez-les-donnees)

[Wiki - Racinisation](https://fr.wikipedia.org/wiki/Racinisation)

Afin de faciliter les choses, on stem avant de bager (Raciniser/Suppr. des stopwords avant de grouper)

[TF-IDF](https://towardsdatascience.com/natural-language-processing-feature-engineering-using-tf-idf-e8b9d00e7e76)

In [7]:
#A activer afin de récupèrer les corpus de NLTK
#Selectionner 'all-corpora'
#import nltk
#nltk.download()

from nltk.corpus import stopwords
sw = stopwords.words("english")
print len(sw)

179


In [13]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")
print stemmer.stem("responsiveness")#Permet d'obtenir le radical d'un mot (voir wiki)
print stemmer.stem("responsivity")
print stemmer.stem("unresponsive")

respons
respons
unrespons


In [2]:
from nltk.stem.snowball import SnowballStemmer
import string

def parseOutText(f):
    """ given an opened email file f, parse out all text below the
        metadata block at the top
        (in Part 2, you will also add stemming capabilities)
        and return a string that contains all the words
        in the email (space-separated) 
        
        example use case:
        f = open("email_file_name.txt", "r")
        text = parseOutText(f)
        
        """


    f.seek(0)  ### go back to beginning of file (annoying)
    all_text = f.read()

    ### split off metadata
    content = all_text.split("X-FileName:")
    
    
    if len(content) > 1:
        ### remove punctuation
        text_string = content[1].translate(string.maketrans("", ""), string.punctuation)
            #Grossière erreur:
            #Lors de la "Traduction/Racinisation", on réalise you're -> youre -> your
            #Alors qu'il faudrait découper la contraction en 'you are' en premier lieu
            #NUL !!!

        ### project part 2: comment out the line below
        #words = text_string

        ### split the text string into individual words, stem each word,
        ### and append the stemmed word to words (make sure there's a single
        ### space between each stemmed word)
        
        stemmer = SnowballStemmer("english")
        words = text_string.split()
        
        words_stemmed = list()
        for word in words:
            words_stemmed.append((stemmer.stem(word)).encode("utf-8"))
            
        sentence = " ".join(words_stemmed)
        
        
    return sentence
    #return words_stemmed
    

def main():
    ff = open("../text_learning/test_email.txt", "r")
    text = parseOutText(ff)
    print text

main()




hi everyon if you can read this messag your proper use parseouttext pleas proceed to the next part of the project


In [8]:
import os
import pickle
import re
import sys

#sys.path.append( "../tools/" )
#from parse_out_email_text import parseOutText

"""
    Starter code to process the emails from Sara and Chris to extract
    the features and get the documents ready for classification.

    The list of all the emails from Sara are in the from_sara list
    likewise for emails from Chris (from_chris)

    The actual documents are in the Enron email dataset, which
    you downloaded/unpacked in Part 0 of the first mini-project. If you have
    not obtained the Enron email corpus, run startup.py in the tools folder.

    The data is stored in lists and packed away in pickle files at the end.
"""


from_sara  = open("../text_learning/from_sara.txt", "r")
from_chris = open("../text_learning/from_chris.txt", "r")

from_data = []
word_data = []

### temp_counter is a way to speed up the development--there are
### thousands of emails from Sara and Chris, so running over all of them
### can take a long time
### temp_counter helps you only look at the first 200 emails in the list so you
### can iterate your modifications quicker
temp_counter = 0


for name, from_person in [("sara", from_sara), ("chris", from_chris)]:
    for path in from_person:
        ### only look at first 200 emails when developing
        ### once everything is working, remove this line to run over full dataset
        #temp_counter += 1
        if temp_counter < 10:
            path = os.path.join('..', path[:-1])
            #print path
            email = open(path, "r")

            ### use parseOutText to extract the text from the opened email
            text = parseOutText(email)
            
            ### use str.replace() to remove any instances of the words
            ### ["sara", "shackleton", "chris", "germani"]
            repl_words = ["sara", "shackleton", "chris", "germani", "sshacklensf", "cgermannsf", "houectect"]
            #Lesson 12: Ajout de deux mots (les signatures) exclus de la base
            for repl_word in repl_words:
                text = text.replace(repl_word, "")

            ### append the text to word_data
            word_data.append(text)

            ### append a 0 to from_data if email is from Sara, and 1 if email is from Chris
            if name == 'sara':
                from_data.append(0)
            elif name == 'chris':
                from_data.append(1)

            email.close()

print "emails processed"
from_sara.close()
from_chris.close()

pickle.dump( word_data, open("your_word_data.pkl", "w") )
pickle.dump( from_data, open("your_email_authors.pkl", "w") )

print word_data[152]



### in Part 4, do TfIdf vectorization here
#Nope.



emails processed
tjonesnsf stephani and sam need nymex calendar


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(word_data)
print(len(vectorizer.get_feature_names()))
print vectorizer.get_feature_names()[34597]


38757
stephaniethank
